In [1]:
HTML("""
<style>
@import url('https://fonts.googleapis.com/css?family=Nanum+Gothic&display=swap');
body {
    font-family:'Nanum Gothic', sans-serif;
}
</style>
"""
)

HTML{String}("<style>\n@import url('https://fonts.googleapis.com/css?family=Nanum+Gothic&display=swap');\nbody {\n    font-family:'Nanum Gothic', sans-serif;\n}\n</style>\n")

In [2]:
using Statistics
using Dates, TimeZones
using CSV, DataFrames, DataFramesMeta
using Plots, Plots.PlotMeasures
using PyPlot
using Formatting
pyplot()

Plots.PyPlotBackend()

In [3]:
nanum12 = Plots.font(family="NanumGothic", pointsize=12)
nanum14 = Plots.font(family="NanumGothic", pointsize=14)
nanum18 = Plots.font(family="NanumGothic", pointsize=18)
nanum24 = Plots.font(family="NanumGothic", pointsize=24)
default(titlefont=nanum24, guidefont=nanum18, tickfont=nanum18, legendfont=nanum18)

In [4]:
stn_codes = [111121, 111123, 111131, 111141, 111142,
    111151, 111152, 111161, 111171, 111181,
    111191, 111201, 111212, 111221, 111231,
    111241, 111251, 111261, 111262, 111273,
    111274, 111281, 111291, 111301, 111311]
stn_names = ["중구", "종로구", "용산구", "광진구", "성동구",
    "중랑구", "동대문구", "성북구", "도봉구", "은평구",
    "서대문구", "마포구", "강서구", "구로구", "영등포구",
    "동작구", "관악구", "강남구", "서초구", "송파구",
    "강동구", "금천구", "강북구", "양천구", "노원구"]

25-element Array{String,1}:
 "중구"  
 "종로구" 
 "용산구" 
 "광진구" 
 "성동구" 
 "중랑구" 
 "동대문구"
 "성북구" 
 "도봉구" 
 "은평구" 
 "서대문구"
 "마포구" 
 "강서구" 
 "구로구" 
 "영등포구"
 "동작구" 
 "관악구" 
 "강남구" 
 "서초구" 
 "송파구" 
 "강동구" 
 "금천구" 
 "강북구" 
 "양천구" 
 "노원구" 

In [59]:
case_name = "test"
zoom_mindate = DateTime(2018, 1, 1, 1)
zoom_maxdate = DateTime(2018, 12, 31, 23)
zoom_sdate = DateTime(2018, 7, 1, 1)
zoom_fdate = DateTime(2018, 7, 10, 23)
zoom_sdate2 = DateTime(2018, 11, 1, 1)
zoom_fdate2 = DateTime(2018, 11, 10, 23)
jongro_stncode = 111123
jongro_stnname = "종로구"
jongro_idx = 2
seongbuk_stncode =  111161
seongbuk_stnname = "성북구"
seongbuk_idx = 8
songpa_stncode = 111273
songpa_stnname = "송파구"
seongbuk_idx = 20
gangseo_stncode = 111212
gangseo_stnname = "강서구"
seongbuk_idx = 13
seongdong_idx = 5

5

In [60]:
jongro_raw_csv = "/home/appleparan/input/jongro_single.csv"
feature_csv = "/home/appleparan/data/" * case_name * "/post/feature/feature_stats.csv"
station_csv = "/home/appleparan/data/" * case_name * "/post/station/station_stats.csv"
forecast_csv = "/home/appleparan/data/" * case_name * "/post/forecast/forecast_stats.csv"
pearson_corr_csv = "/home/appleparan/data/" * case_name * "/pearson_corr.csv"

"/home/appleparan/data/test/pearson_corr.csv"

In [61]:
ENV["GKSwstype"] = "100"
ENV["GKS_ENCODING"] = "utf-8"
const BG_COLOR = RGB(255/255, 255/255, 255/255)
const LN_COLOR = RGB(67/255, 75/255, 86/255)
const MK_COLOR = RGB(67/255, 75/255, 86/255)
const LN01_COLOR = RGB(202/255,0/255,32/255)
const LN02_COLOR = RGB(5/255,113/255,176/255)
const FL01_COLOR = RGB(239/255, 138/255, 98/255)
const FL02_COLOR = RGB(103/255, 169/255, 207/255)
const datefmt = "yyyymmddHH"    

"yyyymmddHH"

# Zoomed Plot

In [62]:
function plot_DNN_lineplot(s_date::DateTime, f_date::DateTime, ycol::Symbol)
    csv_in_name = jongro_raw_csv
    ycol_str = "PM10"
    if ycol == :PM25
        ycol_str = "PM2.5"
    elseif ycol == :PM10
        ycol_str = "PM10"
    end
    #output_dir = "/home/appleparan/data/" * case_name * "/post/"
    output_dir = "/home/appleparan/data/" * case_name * "/post/"
    line_out_path = output_dir *
        string(ycol) * "_" *
        "rawinput_" *
        Dates.format(s_date + Dates.Hour(24), datefmt) * "_"  *
        Dates.format(f_date + Dates.Hour(24), datefmt) 
    
    df_raw = CSV.read(csv_in_name, copycols=true)
    df_raw[:, :date] = DateTime.(ZonedDateTime.(df_raw[:, :date], "yyyy-mm-ddTHH:MM:SS.ssszzzz"))
    df_indates = @linq df_raw |>
        where(:date .>= s_date + Dates.Hour(1) , :date .<= f_date + Dates.Hour(1))
    
    # plot in dates
    gr(size = (2560, 1080))
    pl1 = Plots.plot(df_indates[:, :date], df_indates[:, ycol],
        ylim = (0.0,maximum(df_indates[:, ycol])),
        line=:solid, linewidth=5, label="$(string(ycol))",
        guidefontsize = 18, titlefontsize = 24, tickfontsize = 18, legendfontsize = 18, margin=15px,
        guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
        background_color = BG_COLOR, color=LN01_COLOR,
        title=ycol_str, 
        xlabel="date", ylabel=ycol_str, legend=:top)

    Plots.png(pl1, line_out_path * ".png")
    Plots.svg(pl1, line_out_path * ".svg")

    l = @layout [a ; b]
    #Plots.plot(pl1, pl2, layout = l)
end

plot_DNN_lineplot (generic function with 1 method)

In [68]:
plot_DNN_lineplot( DateTime(2018, 1, 1, 1), DateTime(2018, 12, 31, 23), :PM10)
plot_DNN_lineplot( DateTime(2018, 1, 1, 1), DateTime(2018, 12, 31, 23), :PM25)

Plots.GridLayout(2, 1)

In [69]:
plot_DNN_lineplot( DateTime(2017, 1, 1, 1), DateTime(2017, 12, 31, 23), :PM10)
plot_DNN_lineplot( DateTime(2017, 1, 1, 1), DateTime(2017, 12, 31, 23), :PM25)

Plots.GridLayout(2, 1)

In [70]:
plot_DNN_lineplot( DateTime(2016, 1, 1, 1), DateTime(2016, 12, 31, 23), :PM10)
plot_DNN_lineplot( DateTime(2016, 1, 1, 1), DateTime(2016, 12, 31, 23), :PM25)

Plots.GridLayout(2, 1)

In [71]:
plot_DNN_lineplot( DateTime(2015, 1, 1, 1), DateTime(2015, 12, 31, 23), :PM10)
plot_DNN_lineplot( DateTime(2015, 1, 1, 1), DateTime(2015, 12, 31, 23), :PM25)

Plots.GridLayout(2, 1)

In [72]:
plot_DNN_lineplot( DateTime(2014, 1, 1, 1), DateTime(2014, 12, 31, 23), :PM10)
plot_DNN_lineplot( DateTime(2014, 1, 1, 1), DateTime(2014, 12, 31, 23), :PM25)

Plots.GridLayout(2, 1)